## Analyzing Slovakia's population count changes between 2006 and 2021 👪

#### Goal: 
to describe Slovakia's current (2021) population count and its changes between 2006 and 2021

#### Data:
- GEOSTAT (2006) and Eurostat Census Grid (2021) population data (1x1 km grid) available at: https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/population-distribution-demography/geostat

#### Research area: 
Slovakia, EU

#### Processing steps:
0. Importing packages
1. Scraping, downloading and unzipping data
2. Creating standardized population data for a single year
3. Filtering Slovakia's grids from all-EU data (only for 2006)
4. Performing a spatial join between all-EU data for 2021 and Slovakia's data for 2006 => merged dataset limited to Slovakia's grids
5. Finding 5 grids characterized by the highest population increase
6. Reprojecting population data to WGS84 (geographic coordinate system)
7. Calculating Slovakia's 2021 population count statistics

Prepared by: Aleksandra Radecka <br>
e-mail: aleksandraradecka@protonmail.com <br>
LinkedIn: https://www.linkedin.com/in/aleksandraradecka/

## Step 0: importing packages

In [3]:
import glob
import io

from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd
import requests
import zipfile  

## Step 1: scraping, downloading and unzipping data

In [1]:
def download_geostat_zip(input_url, year):
    """
    Scrapes the website to find and download and unzip a zip file containg statistics for a given year
    :param input_url: str
        url to website to scrape
    :param year: str
        year of measurement 
    :return: tuple of str
        measurement year and name of the folder containg unzipped data
    """    
    # sending request and parsing obtained content
    response = requests.get(input_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # finding links and filtering them out by format and measurement year
    links = soup.find_all('a')
    zip_year_link = [link.get('href') for link in links if link.get('href') and link.get('href').endswith('.zip') and year in link.get('href')]
    # establishing the zip's url and downloading it
    zip_url = zip_year_link[0] if zip_year_link[0].startswith('http') else f"https://ec.europa.eu{zip_year_link[0]}"
    zip_response = requests.get(zip_url)
    filename = zip_url.split('/')[-1]
    with open(filename, 'wb') as f:
        f.write(zip_response.content)
        print(f"{filename} downloaded successfully")
    # unzipping the file 
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(filename[:-4])
        print(f"{filename} unzipped successfully")
    return(year, filename[:-4])

In [2]:
y1, folder1 = download_geostat_zip(input_url = 'https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/population-distribution-demography/geostat', year = '2006')

GEOSTAT_Grid_POP_2006_1K.zip downloaded successfully
GEOSTAT_Grid_POP_2006_1K.zip unzipped successfully


In [3]:
y2, folder2 = download_geostat_zip(input_url = 'https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/population-distribution-demography/geostat', year = '2021')

Eurostat_Census-GRID_2021_V1-0.zip downloaded successfully
Eurostat_Census-GRID_2021_V1-0.zip unzipped successfully


## Step 2: creating standardized population data for a single year 

In [4]:
def prepare_geostat(input_folder, year):
    """
    Prepares standardized population data for a given year 
    :param input_folder: str
        name of the folder containg unzipped geostat data
    :param year: str
        year of measurement 
    :return: gdf 
        prepared (standardized) geostat GeoPandas dataframe for a given year 
    """
    # finding a vector file
    vector_files = []
    for ext in ['.geojson', '.shp', 'gpkg']:
        vector_files.extend(glob.glob(input_folder + '/**/*' + ext, recursive=True)) 
    gdf = gpd.read_file(vector_files[0])
   
    # possible finding of a csv file
    csv_files = glob.glob(input_folder + '/**/*.csv', recursive=True)
    if len(csv_files) == 1:
        df = pd.read_csv(csv_files[0], sep = ';')
        # standardizing grid ID column name
        for data in [gdf, df]:
            columns_with_grd = [col for col in data.columns if 'GRD' in col]
            data.rename(columns={columns_with_grd[0]: 'GRD_ID'}, inplace=True)
        # joining tabular data to vector data
        pop = gdf.merge(df, on = 'GRD_ID')
    else:
        pop = gdf
    # standardizing population column name
    columns_with_value = [col for col in pop.columns if 'POP' in col or 'OBS' in col]
    pop.rename(columns={columns_with_value[0]: 'pop_' + year}, inplace=True)
    # changing geometry to centroids
    pop['geometry'] = pop['geometry'].centroid
    return(pop)

In [6]:
pop_2006 = prepare_geostat(input_folder = folder1, year = y1)

In [7]:
pop_2021 = prepare_geostat(input_folder = folder2, year = y2)

## Step 3: filtering Slovakia's grids from all-EU data (only for 2006)

In [8]:
def filter_country(pop, country_code):
    """
    Filters out population data for a given country (without boarder grids) from data for a given year
    :param pop_gdf: GeoPandas dataframe
        population data for a given year
    :param country_code: str
        country_code according to ISO 3166
    :return: GeoPandas dataframe
         population data for a given country and year
    """
    # indexing gdf
    pop_country_gdf = pop.loc[pop['CNTR_CODE'] == country_code, :]
    return(pop_country_gdf)

In [9]:
pop_sk_2006 = filter_country(pop = pop_2006, country_code = 'SK')

<u>Comments</u>:
- To make it easier for the user, a dictionary of country names and their codes could be created. I would allow the user to specify only the name of the country without having to remember/check the country's ISO code.
- I decided that only "pure" Slovakian grids should be used, meaning that the boarder grids were neglected in my approach. This is because year 2006 included data on the fractional population numbers for the boarder grids, while year 2021 didn't. Comparing the two years in the later stage (when calculating relative difference) would result in errors. 

## Step 4: performing a spatial join between all-EU data for 2021 and Slovakia's data for 2006 => merged dataset limited to Slovakia's grids

In [10]:
def join_pair_years(pop_1, pop_2):
    """
    Merges population data from one year to the data for the second using spatial join operation
    :param pop_1, pop_2: GeoPandas dataframe
        population data for a given year
    :return: GeoPandas dataframe
         population data for a given country and years
    """
    # performing spatial join
    pop_2years_gdf = pop_1.sjoin_nearest(pop_2)
    return(pop_2years_gdf)

In [11]:
pop_sk_2006_2021 = join_pair_years(pop_1 = pop_sk_2006, pop_2 = pop_2021)

<u>Comments</u>:
- Spatial join was chosen as a way to merge the two datasets as 1. both had different grid IDs 2. 2021 data were not supplied with *"country_code"* attribute
- Naturally, the function could be extended to merge more than two gdfs

## Step 5: finding 5 grids characterized by the highest population increase

In [12]:
def find_rel_increase(pop_data, pop_field_year_1, pop_field_year_2, n_grids):
    """
    Limits population data to n grids with the highest relative increase between year 1 and year 2
    :param pop_data: GeoPandas dataframe
        population data
    :param pop_field_year_1: str
        name of the column containing population data for the first year
    :param pop_field_year_2: str
        name of the column containing population data for the second year
    :param n_grids: int
        number of grids with the highest relative increase   
    :return: GeoPandas dataframe
         population data for n selected grids
    """
    # calculating the relative change
    pop_data['rel_change'] = (pop_data[pop_field_year_2] - pop_data[pop_field_year_1]) / pop_data[pop_field_year_1]
    # finding n grids with the highest value 
    pop_data_top_5 = pop_data.nlargest(n_grids, 'rel_change')
    return(pop_data_top_5)

In [13]:
pop_sk_5top = find_rel_increase(pop_data = pop_sk_2006_2021, pop_field_year_1 = 'pop_2006', pop_field_year_2 = 'pop_2021', n_grids = 5)

In [14]:
pop_sk_5top

,GRD_ID_left,geometry,pop_2006,YEAR,METHD_CL,CNTR_CODE,DATA_SRC,index_right,GRD_ID_right,pop_2021,rel_change
1458045,1kmN2815E4857,POINT (4857500.000 2815500.000),3,2006,D,SK,AIT,2142514,CRS3035RES1000mN2815000E4857000,1059.0,352.0
1508183,1kmN2771E4931,POINT (4931500.000 2771500.000),1,2006,D,SK,AIT,2039415,CRS3035RES1000mN2771000E4931000,341.0,340.0
1459443,1kmN2803E4859,POINT (4859500.000 2803500.000),1,2006,D,SK,AIT,2114276,CRS3035RES1000mN2803000E4859000,305.0,304.0
1458046,1kmN2816E4857,POINT (4857500.000 2816500.000),4,2006,D,SK,AIT,2144875,CRS3035RES1000mN2816000E4857000,1212.0,302.0
1463616,1kmN2814E4865,POINT (4865500.000 2814500.000),2,2006,D,SK,AIT,2140162,CRS3035RES1000mN2814000E4865000,466.0,232.0


<u>Comments</u>:
- Instead of inserting whole field names, only the years could be used as the field names were standardized in the step above. 

## Step 6: reprojecting population data to WGS84 (geographic coordinate system)

In [15]:
def reproject_vector(input_data, output_epsg):
    """
    Reproject data from one EPSG to another
    :param input_data: GeoPandas dataframe
        vector data
    :param output_epsg: str
        EPSG code  
    :return: GeoPandas dataframe
         reprojected vector data
    """
    # performing reprojection
    reprojected_data = input_data.to_crs(epsg = output_epsg)
    return(reprojected_data)

In [16]:
pop_sk_top5_geo = reproject_vector(input_data = pop_sk_5top, output_epsg = '4326')

<u>Comments</u>:
- I performed reprojection as the further tasks required geographical lat and lon as part of the output (attachment 1), not cartographical x and y. 

In [17]:
pop_sk_top5_geo.to_csv('attachment_1.csv')

## Step 7: calculating Slovakia's 2021 population count statistics

In [28]:
def calculate_pop_stats(pop_data, pop_field):
    """
    Calculates statistics (avg and median) for a selected population year
    :param pop_data: GeoPandas dataframe
        population data
    :param pop_field: str
        name of the column containing population data for the analyzed year
    :return: Pandas dataframe
         population statistics
    """
    # creating empty df
    stats_df = pd.DataFrame(columns = ['avg', 'median'], index = ['pop_stats'])
    # calculating stats
    stats_df.loc['pop_stats', 'avg'] = pop_data[pop_field].mean()
    stats_df.loc['pop_stats', 'median'] = pop_data[pop_field].median()
    return(stats_df)

In [29]:
sk_2021_stats = calculate_pop_stats(pop_data = pop_sk_2006_2021, pop_field = 'pop_2021')

In [30]:
sk_2021_stats

,avg,median
pop_stats,365.010984,105.0


In [31]:
sk_2021_stats.to_csv('attachment_2.csv')

<u>Final comments</u>:
- It would be a good idea to supply the code with tests that check e.g. the completeness of data, type and range of values for population data, the alignment of grids included in each year of measurements or the match between CRS. 
- Some years of measurement included data in an additional, raster format. While raster data are lighter (easier to import) and easier to parallelize (important when processing large amounts of data), vector data having all attributes inside are easier to index and transform to .csv output files. 